In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from shapely.geometry import Polygon
from scipy.spatial.distance import directed_hausdorff

In [2]:
original = pd.read_csv('./LSTM_TrajGAN/data/test_latlon.csv')
syn_best = pd.read_csv('ForMarc/traj_0306_9_total_latlon.csv')
syn_demo = pd.read_csv('./LSTM_TrajGAN/results/syn_traj_test.csv')

In [3]:
# IoU based on Convex Hull
syn_best_IOU = []
syn_demo_IOU = []
# Hausdorff
syn_best_Haus = []
syn_demo_Haus = []

tidlist = list(original['tid'].unique())

for tid in tidlist:
    real_traj = np.concatenate([np.array(original.loc[original['tid'] == tid]["lat"]).reshape(-1,1),
                                np.array(original.loc[original['tid'] == tid]["lon"]).reshape(-1,1)],
                               axis=1)
    syn_best_traj = np.concatenate([np.array(syn_best.loc[syn_best['tid'] == tid]["lat"]).reshape(-1,1),
                               np.array(syn_best.loc[syn_best['tid'] == tid]["lon"]).reshape(-1,1)],
                              axis=1)
    syn_demo_traj = np.concatenate([np.array(syn_demo.loc[syn_demo['tid'] == tid]["lat"]).reshape(-1,1),
                               np.array(syn_demo.loc[syn_demo['tid'] == tid]["lon"]).reshape(-1,1)],
                              axis=1)
    
    syn_best_Haus.append(directed_hausdorff(real_traj,syn_best_traj)[0])
    syn_demo_Haus.append(directed_hausdorff(real_traj,syn_demo_traj)[0])
    
    real_traj_convex_hull = Polygon(real_traj).convex_hull
    syn_best_traj_convex_hull = Polygon(syn_best_traj).convex_hull
    syn_demo_traj_convex_hull = Polygon(syn_demo_traj).convex_hull
    syn_best_I = real_traj_convex_hull.intersection(syn_best_traj_convex_hull).area
    syn_best_U = real_traj_convex_hull.union(syn_best_traj_convex_hull).area
    syn_best_IOU.append(syn_best_I/syn_best_U)
    syn_demo_I = real_traj_convex_hull.intersection(syn_demo_traj_convex_hull).area
    syn_demo_U = real_traj_convex_hull.union(syn_demo_traj_convex_hull).area
    syn_demo_IOU.append(syn_demo_I/syn_demo_U)

In [4]:
df = pd.DataFrame({'syn_best_IOU':syn_best_IOU,
                   'syn_demo_IOU':syn_demo_IOU,
                   'syn_best_Haus':syn_best_Haus,
                   'syn_demo_Haus':syn_demo_Haus})
df.describe()

,syn_best_IOU,syn_demo_IOU,syn_best_Haus,syn_demo_Haus
count,1027.000000,1027.000000,1027.000000,1027.000000
mean,0.575889,0.356594,0.012179,0.022158
std,0.236035,0.245971,0.006732,0.009243
min,0.000000,0.000000,0.000944,0.004016
25%,0.432254,0.144021,0.007788,0.016220
50%,0.634701,0.343863,0.010581,0.020097
75%,0.764514,0.562463,0.014937,0.025622
max,0.944604,0.938343,0.046467,0.077150
